# AFM

<img src="../img/afm.png">

In [1]:
import pandas as pd
import numpy as np
import json
import torch
from torch import nn
from torch.utils.data.dataset import Dataset
from collections import OrderedDict
from itertools import combinations

In [2]:
learning_rate = 0.0001
batch_size = 64
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
with open('../data/movielens/feature_map.json') as obj:
    feature_map = json.load(obj)
feature_map

{'dataset_id': 'movielens',
 'num_fields': 26,
 'feature_specs': {'movieId': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 935,
   'index': 0},
  'userId': {'source': 'user',
   'type': 'categorical',
   'vocab_size': 22540,
   'index': 1},
  'rating': {'source': 'user',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 2},
  'timestamp': {'source': 'user',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 3},
  'releaseYear': {'source': 'item',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 4},
  'movieGenre1': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 18,
   'index': 5},
  'movieGenre2': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 18,
   'index': 6},
  'movieGenre3': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 15,
   'index': 7},
  'movieRatingCount': {'source': 'item',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 8},
  'movieAvgRating': {'source': 'item',
   'type': 'numerical',
 

In [4]:
class MovielensDataset(Dataset):
    def __init__(self, url):
        self.df = pd.read_csv(url)
    
    def __getitem__(self, idx):
        x, y = self.df.iloc[idx, :-1].values.astype(np.float32), self.df.iloc[idx, -1].astype(np.float32)
        return x, y
    
    def __len__(self):
        return self.df.shape[0]

In [5]:
train_dataset = MovielensDataset('../data/movielens/data_for_train.csv')
test_dataset = MovielensDataset('../data/movielens/data_for_test.csv')

In [6]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [7]:
class AFM(nn.Module):
    def __init__(self,
                 feature_map,
                 embedding_dim=10,
                 attention_dim=10):
        super(AFM, self).__init__()
        self.feature_map = feature_map
        # Embedding
        self.embedding = nn.ModuleDict()
        for feature, feature_spec in feature_map['feature_specs'].items():
            if feature_spec['type'] == 'numerical':
                self.embedding[feature] = nn.Linear(1, embedding_dim, bias=False)
            elif feature_spec['type'] == 'categorical':
                padding_idx = feature_spec.get('padding_idx', None)
                self.embedding[feature] = nn.Embedding(feature_spec['vocab_size'],
                                                       embedding_dim,
                                                       padding_idx=padding_idx)
        # LR
        self.batch_norm = nn.BatchNorm1d(feature_map['num_fields'])
        self.lr = nn.Linear(feature_map['num_fields'], 1)
        # Product Layer
        self.product_layer = PairwiseInteraction(feature_map['num_fields'])
        # Attention Net
        self.attention_net = AttentionNet(embedding_dim, attention_dim)
        # p
        self.p = nn.Linear(embedding_dim, 1, bias=False)
        # Sigmoid
        self.output_activation = nn.Sigmoid()

    def forward(self, X):
        feature_emb_list = []
        for feature, feature_spec in self.feature_map['feature_specs'].items():
            if feature_spec['type'] == 'numerical':
                raw_feature = X[:, feature_spec['index']].float().view(-1, 1)
            elif feature_spec['type'] == 'categorical':
                raw_feature = X[:, feature_spec['index']].long()
            embedding_vec = self.embedding[feature](raw_feature)
            feature_emb_list.append(embedding_vec)
        feature_emb = torch.stack(feature_emb_list, dim=1)
        flat_feature_emb = feature_emb.flatten(start_dim=1)
        lr_out = self.lr(self.batch_norm(X))
        elementwise_product = self.product_layer(feature_emb) # bs * f(f-1)/2 * embedding_dim
        attention_weight = self.attention_net(elementwise_product)
        attention_sum = torch.sum(attention_weight * elementwise_product, dim=1)
        afm_out = self.p(attention_sum)
        out = lr_out + afm_out
        y_pred = self.output_activation(out).squeeze(1)
        return y_pred

class PairwiseInteraction(nn.Module):
    def __init__(self, num_fields):
        super(PairwiseInteraction, self).__init__()
        p, q = zip(*list(combinations(range(num_fields), 2)))
        self.field_p = nn.Parameter(torch.LongTensor(p), requires_grad=False)
        self.field_q = nn.Parameter(torch.LongTensor(q), requires_grad=False)
       
    def forward(self, feature_emb):
        V_i = torch.index_select(feature_emb, 1, self.field_p)
        V_j = torch.index_select(feature_emb, 1, self.field_q)
        return V_i * V_j

class AttentionNet(nn.Module):
    def __init__(self,
                 input_dim,
                 output_dim):
        super(AttentionNet, self).__init__()
        self.W = nn.Parameter(torch.Tensor(input_dim, output_dim))
        self.b = nn.Parameter(torch.Tensor(output_dim))
        self.relu = nn.ReLU()
        self.h = nn.Parameter(torch.Tensor(output_dim, 1))
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        out = X @ self.W + self.b
        out = self.relu(out)
        out = out @ self.h
        a = self.softmax(out)
        return a

In [8]:
model = AFM(feature_map).to(device)
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (X, y) in enumerate(train_loader):
        X = X.to(device)
        y = y.to(device)

        # Forward pass
        output = model(X)
        loss = criterion(output, y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 300 == 0:
            print("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}".format(
                epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/5], Step [300/1388] Loss: 0.6543
Epoch [1/5], Step [600/1388] Loss: 0.6257
Epoch [1/5], Step [900/1388] Loss: 0.5844
Epoch [1/5], Step [1200/1388] Loss: 0.6716
Epoch [2/5], Step [300/1388] Loss: 0.6212
Epoch [2/5], Step [600/1388] Loss: 0.5338
Epoch [2/5], Step [900/1388] Loss: 0.5909
Epoch [2/5], Step [1200/1388] Loss: 0.5086
Epoch [3/5], Step [300/1388] Loss: 0.5143
Epoch [3/5], Step [600/1388] Loss: 0.4914
Epoch [3/5], Step [900/1388] Loss: 0.4611
Epoch [3/5], Step [1200/1388] Loss: 0.4348
Epoch [4/5], Step [300/1388] Loss: 0.4116
Epoch [4/5], Step [600/1388] Loss: 0.4403
Epoch [4/5], Step [900/1388] Loss: 0.4128
Epoch [4/5], Step [1200/1388] Loss: 0.4116
Epoch [5/5], Step [300/1388] Loss: 0.3725
Epoch [5/5], Step [600/1388] Loss: 0.3636
Epoch [5/5], Step [900/1388] Loss: 0.3027
Epoch [5/5], Step [1200/1388] Loss: 0.3198


In [9]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device).bool()
        output = model(X)
        y_pred = output > 0.5
        total += y.shape[0]
        correct += (y_pred == y).sum().item()

    print('Accuracy of the model on the test images: {:.2f} %'.format(
        100 * correct / total))

Accuracy of the model on the test images: 89.74 %
